<a href="https://colab.research.google.com/github/robertonetti/small-RNA-notebook/blob/main/notebook_smallRNA_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload and unzip data

In [1]:
# Create the destination folder
destination_folder = "/content/"
mkpath(destination_folder)

# Use the system's 'unzip' command
# -o = overwrite files without asking
# -d = specify destination directory
zip_file = "/content/data.zip"

run(`unzip -o $zip_file -d $destination_folder`)

println("Decompression completed in: ", destination_folder)

Archive:  /content/data.zip
   creating: /content/data/
  inflating: /content/__MACOSX/._data  
  inflating: /content/data/fitness_DMS_2.txt  
  inflating: /content/__MACOSX/data/._fitness_DMS_2.txt  
  inflating: /content/data/.DS_Store  
  inflating: /content/__MACOSX/data/._.DS_Store  
  inflating: /content/data/train_sequences_2.fasta  
  inflating: /content/__MACOSX/data/._train_sequences_2.fasta  
  inflating: /content/data/train_sequences_1.fasta  
  inflating: /content/__MACOSX/data/._train_sequences_1.fasta  
  inflating: /content/data/DMS_2.fasta  
  inflating: /content/__MACOSX/data/._DMS_2.fasta  
  inflating: /content/data/structures_1.txt  
  inflating: /content/__MACOSX/data/._structures_1.txt  
  inflating: /content/data/structures_2.txt  
  inflating: /content/__MACOSX/data/._structures_2.txt  
  inflating: /content/data/sequences_to_assign.fasta  
  inflating: /content/__MACOSX/data/._sequences_to_assign.fasta  
  inflating: /content/data/structures.svg  
  inflating:

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = auto


# Add Packages

In [ ]:
using Pkg; Pkg.add(url="https://github.com/FrancescoCalvanese/FCSeqTools.jl"); Pkg.add("StatsBase"), Pkg.add("PyCall"),  Pkg.add("PyPlot")
using FCSeqTools, StatsBase, Statistics, LinearAlgebra, PyCall, PyPlot;
include("utils.jl");
using .utils;

---
# **Section 1:**  Training an eaDCA Model on RNA Structures

## We will begin by training an eaDCA model on a specific RNA family, which is known to fold into a **internal loop** structure. After conducting some analysis on the internal loop structure, we will proceed to another structure called the **hairpin**.


structures.svg

## Some useful functions:

``` julia
data = read_fasta(file_path::String)
"""
Parameters:
file_path: The path to the FASTA file (use " ").

Returns:
data: A matrix containing the sequences represented as Potts spins.
"""
```

```julia
results = eaDCA(q::Int, lambda::Float, nchains::Int, data::Matrix{Int}, output_file::String)
"""
Train Potts model with edge activation algorithm.

Parameters:
q           : The number of nucleotides (use 4).
lambda      : The regularization parameter (use 0.1).
nchains     : The number of sequences used by the algorithm to train the model (use 7,000).
data        : The matrix containing the training data.
output_file : The name of the output file where results will be saved. (example: eaDCA_output.txt)

Returns:
score           : A vector depicting the scores acquired throughout the training phases.
likelihood_gain : A vector showcasing the likelihood gains acquired during the training phases.
sequences       : The artificial sequences generated by the model.
J               : A matrix representing the coupling within the model.
h               : A vector containing the local fields learned by the model.
contact_list    : The contact list derived from the learned graph.
site_degree     : The site degree derived from the learned graph.
edge_list       : The edge list derived from the learned graph.
"""
```
you can access the **generated sequences**, **J** and **h** as:
```julia
sequences = results[3]
J = results[4]
h = results[5]
```

---
## **Task 1:** Train the eaDCA model on the Internal Loop RNA family.

The file "/content/data/train_sequences_1.fasta" contains natural sequences for the Internal Loop RNA. Use the ```read_fasta()``` function to load the data and ```eaDCA()``` to train a model. Extract the **generated sequences**, the coupling matrix **J**, and the local fields **h**.

In [ ]:
# set some parameters
q = 4 # number of nucleotides
L = 30 # length of the sequences
lambda = 0.1 # regularization
nchains = 7000 # number of sequences for training
output_file_intloop = "output_intloop.txt" ;

In [ ]:
data_intloop = read_fasta("/content/data/train_sequences_1.fasta")
results_intloop = eaDCA(q, lambda, nchains, data_intloop, output_file_intloop);

In [ ]:
gen_sequences_intloop = results_intloop[3]
J_intloop = results_intloop[4]
h_intloop = results_intloop[5];


---
## **Task 2**: Demonstrate that the model's energy landscape accurately represents the natural sequences.

Use the ```sequences_energy()``` function to calculate the energies of both the natural and generated sequences. Compare their energy distributions using the histogram below. Do they match?

```julia
energies = sequences_energy(data::Matrix{Int}, q::Int, h::Vector{Float}, J::Matrix{Float})
"""
Parameters:
- data: A matrix of training data, where each row represents a sequence.
- q: An integer representing the number of nucleotides.
- h: A vector of floats representing the local fields learned by the model.
- J: A matrix of floats representing the coupling between elements in the model.

Output:
- energies: A vector of floats containing the energy values for each sequence in the dataset.
"""
```




In [ ]:
energy_nat_intloop = sequences_energy(data_intloop, q, h_intloop, J_intloop)
energy_gen_intloop = sequences_energy(gen_sequences_intloop, q, h_intloop, J_intloop);

In [ ]:
# histogram
num_bins = 40
plt.figure(figsize=(10, 5))
plt.hist(energy_nat_intloop, label="natural sequences", bins=num_bins, alpha=0.5, color="blue", edgecolor="black", density=true)
plt.hist(energy_gen_intloop, label="generated sequences", bins=num_bins, alpha=0.5, color="green", edgecolor="black", density=true)
plt.title("Compare Energy")
plt.xlabel("energy")
plt.ylabel("counts")
plt.legend()
plt.show()

# **Section 2:** Secondary Structure Prediction  
---  
## **Task 1:** Visualize the secondary structure of the Internal Loop RNA family  

The file "/content/data/structures_1.txt" contains the **TRUE secondary structure** of the Internal Loop RNA. Use the ```read_secondary_structure()``` function to load the file, then plot it using the code provided below.


```julia
contact_matrix = read_secondary_structure(file_path::String)
"""
Read the secondary structure information from a file.

Parameters:
- file_path: Path to the file containing the secondary structure information (use " ").

Output:
- contact_matrix: Matrix where each element (i, j) contains either 1 or 0, indicating whether residues i and j form a contact or not.
"""

```


In [ ]:
# read contact map from file
contact_matrix_intloop = read_secondary_strucuture("/content/data/structures_1.txt")

# plot contacts
contacts = findall(x -> x .> 0, contact_matrix_intloop)
x = getindex.(contacts, 1)
y = getindex.(contacts, 2)
plt.figure(figsize=(7, 7))
plt.scatter(y, x, label="contacts", color="blue")
plt.xlim(0, 31)
plt.ylim(0, 31)
plt.title("Internal Loop Secondary Structure")
plt.xlabel("Nucleotide Positions")
plt.ylabel("Nucleotide Positions")
plt.grid()
plt.legend()
plt.show()

---  
## **Task 2:** Predict the contacts of the Internal Loop RNA using the DCA model and compare them to the true contacts.  

Use the ```frobenius_norm()``` function to calculate the norm of the coupling matrix **J**. Then, use the ```plot_contact_map()``` function to visualize the predicted contact map and compare it to the true contact map.


```julia
J_normalized = frobenius_norm(J::Matrix{Float}, q::Int, L::Int)
"""
Calculate the Frobenius norm of the coupling matrix.

Parameters:
- J: Coupling matrix representing pairwise interactions.
- q: Number of nucleotides.
- L: Length of the sequence.

Output:
- J_normalized: LxL matrix containing the Frobenius norm of the coupling matrix.
"""
```

```julia
plot_contact_map(predicted_contact_matrix::Matrix{Float}, true_contact_matrix::Matrix{Int}, threshold::Float)
"""
Plot the contact map based on predicted and true contact matrices.

Parameters:
- predicted_contact_matrix: LxL matrix containing predicted contact values.
- true_contact_matrix: True contact matrix.
- threshold: Threshold value for cutting off the predicted contact matrix (try with 1).
"""
```

In [ ]:
threshold = 8
J_intloop_normalized = frobenius_norm(J_intloop, q, L)
plot_contact_map(J_intloop_normalized, contact_matrix_intloop, threshold)

## - A link $(i, j)$ is considered a predicted contact only if $J_{ij}^{norm} \geq \text{threshold}$. Adjust the **threshold** to improve the clarity of the secondary structure signal.  
## - Why do you think the pairings **(6,25)** and **(9,22)** are so challenging to distinguish from the true contacts?

# **Section 3:** Predicting Fitness in a *Deep Mutational Scan* (DMS)  

## Exploring another RNA family: the **hairpin**  

> In the *data* folder, you’ll find several files:  
> - Training sequences: "/content/data/train_sequences_2.fasta" (natural hairpin sequences).  
> - DMS sequences: "/content/data/DMS_2.fasta" (mutated sequences for prediction).  
> - Experimental fitness: "/content/data/fitness_DMS_2.txt" (measured fitness values for the DMS sequences).  

---  
## **Task 1:** How can we predict fitness outcomes accurately in a DMS experiment?  

Use the ```read_fitness()``` function to load the fitness file and ```read_fasta()``` to load the DMS sequences. Train a model on the new hairpin dataset, then use **J** and **h** from the model to predict the fitness of the DMS sequences. (Hint: you can do this easily with one of the previously used functions!). Finally, plot the true fitness against your predictions using ```plot_DMS_prediction()```. Discuss the results.
```julia
fitness = read_fitness(file_path::String)
"""
Read fitness values from a .txt file.

Parameters:
- file_path: Path to the .txt file containing the fitness values.

Output:
- fitness: Vector containing the fitness values.
"""
```


```julia
plot_DMS_prediction(fitness, score_vector)
"""
Plot experimental fitness against prediction scores for each mutation.

Parameters:
- fitness: Vector containing the experimental fitness of each mutation.
- score_vector: Vector containing the prediction score for each mutation.
"""

```


In [ ]:
data_hirpin = read_fasta("/content/data/train_sequences_2.fasta")
data_DMS = read_fasta("/content/data/DMS_2.fasta")
fitness_DMS = read_fitness( "/content/data/fitness_DMS_2.txt")

output_file_hirpin = "output_hirpin.txt"
results_hirpin = eaDCA(q, lambda, nchains, data_hirpin, output_file_hirpin);

In [ ]:
gen_sequences_hirpin = results_hirpin[3]
J_hirpin = results_hirpin[4]
h_hirpin = results_hirpin[5]
energy_DMS_hirpin = sequences_energy(data_DMS, q, h_hirpin, J_hirpin);

In [ ]:
plot_DMS_prediction(fitness_DMS, energy_DMS_hirpin)

# **(BONUS) Section 4:** Family assignment

### Considering the two RNA families we've examined (*hairpin* and *interior loop*), we aim to classify a generic sequence into one of these two families. How can we tackle this challenge?

---
# **Task 1:** perform a logistic regression and compute the accuracy.


> - File with sequences to assign and true label: "/content/data/sequences_to_assign.fasta"




```julia
sequences, labels = read_sequences_and_labels(file_path::String)
"""
Read sequences and their corresponding labels from a file.

Parameters:
- file_path: Path to the file containing sequences and labels.

Returns:
- sequences: Vector of sequences read from the file.
- labels: Vector of labels corresponding to each sequence.
"""
```



In [ ]:
# assign the corresponding variable
data_hirpin = data_hirpin
data_intloop = data_intloop
sequences_to_assign, labels = read_sequences_and_labels("/content/data/sequences_to_assign.fasta");

In [ ]:
@pyimport sklearn.linear_model as lm
@pyimport sklearn.metrics as metrics


LR = lm.LogisticRegression()

# create dataset containing sequences of family 1 and 2
data_train = vcat(data_intloop, data_hirpin)
# define the labels
labels_tot = ones(size(data_train, 1))
labels_tot[size(data_hirpin, 1) + 1 : end] .+= 1;

# fit logistic regression
LR.fit(one_hot_encoding(data_train, q), labels_tot)
LR_prediction = LR.predict(one_hot_encoding(sequences_to_assign, q))

# compute the score of the logistic regression
metrics.accuracy_score(LR_prediction, labels)

---
## **Task 2:** Determine a method to assign labels to the sequences utilizing the trained models and compute its accuracy.
Hint: Plot the histogram of the energies for the sequences to assign, computed with one of the two models. What do you observe? How can you differentiate between hairpin and internal loop RNA?

In [ ]:
energy_seqtossign = sequences_energy(sequences_to_assign, q, h_hirpin, J_hirpin);

In [ ]:
# histogram
num_bins = 40
plt.figure(figsize=(10, 5))
plt.hist(energy_seqtossign, label="sequences to assign", bins=num_bins, alpha=0.5, color="blue", edgecolor="black", density=true)
plt.title("Energy computed with Hirpin model")
plt.xlabel("energy")
plt.ylabel("counts")
plt.legend()
plt.show()

In [ ]:
N = size(sequences_to_assign, 1) # number of sequences to assign
DCA_prediction = zeros(N)

for i in 1:N
  if energy_seqtossign[i] <= 45
    DCA_prediction[i] = 2
  else
    DCA_prediction[i] = 1
  end
end

metrics.accuracy_score(DCA_prediction, labels)

---
# **Mutable sites:** Mutations tend to be more harmful in areas with exceptionally low mutability. A proxy for the mutability of a sequence is represented by the model entropy.


```julia
ci_entropy = context_independent_entropy(data, q)

"""
Compute context-independent entropy for given dataset.

Parameters:
- data: Input data for entropy calculation.
- q: Number of nucleotides.

Returns:
- ci_entropy: Context-independent entropy.
"""

```

```julia
cd_entropy = context_dependent_entropy(sequence::Vector, L::Int, q::Int, J::Matrix{Float}, h::Vector{Float})
"""
Compute context-dependent entropy for a given sequence.

Parameters:
- sequence: Vector representing the sequence.
- L: Length of the sequence.
- q: Number of nucleotides.
- J: Coupling matrix representing pairwise interactions.
- h: Vector of local fields.

Returns:
- cd_entropy: Vector containing context-dependent entropy for each position in the sequence.
"""
```

In [ ]:
# set J_2 and h_2 as the parameters of the Internal Loop RNA model
J_hirpin = J_hirpin
h_hirpin = h_hirpin;

In [ ]:
# compute context INDEPENDENT entropy from the data
CIE = context_independent_entropy(data_hirpin, q)

plt.bar(1:L, CIE)
plt.title("Contex Independent Entropy")
plt.xlabel("Nucleotide Position")
plt.ylabel("Entropy");

In [ ]:
# compute context DEPENDENT entropy from the model
average_CDE = zeros(Float64, L)
for i in 1:size(data_intloop, 1)
  average_CDE  .+= context_dependent_entropy(data_hirpin[i, :], L, q, J_hirpin, h_hirpin)
end

average_CDE = average_CDE ./ size(data_intloop, 1)
plt.bar(1:L, average_CDE)
plt.title("Contex Dependent Entropy")
plt.xlabel("Nucleotide Position")
plt.ylabel("Entropy");

In [ ]:
plt.scatter(CIE, average_CDE);
plt.title("Context Dependent/Independent Entropy")
plt.xlabel("CIE");
plt.ylabel("CDE");
plt.grid();

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />